<a href="https://colab.research.google.com/github/mbenedicto99/TrilhaMLOps/blob/main/ANN_ONNX_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🔧 RNA em PyTorch + Exportação/Inferência ONNX — Notebook Colab
Este notebook reúne e organiza os scripts fornecidos (`data_utils.py`, `model.py`, `train.py`, `export_onnx.py`, `infer.py`, `infer_onnx.py`) para rodar no Google Colab, com etapas claras de **setup**, **treino**, **exportação para ONNX** e **inferência (PyTorch e ONNX)**.

> Dica: execute as células na ordem.


## 1) Ambiente e dependências

In [1]:
# Verificar versão do Python e GPU
import sys, platform, torch

print("Python:", sys.version)
print("Plataforma:", platform.platform())
print("CUDA disponível:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

Python: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
Plataforma: Linux-6.1.123+-x86_64-with-glibc2.35
CUDA disponível: True
GPU: Tesla T4


In [2]:
# Instalar dependências (ajuste conforme necessário)
# Obs.: Colab já traz muitas libs; caso falte algo do seu projeto, inclua aqui.
!pip -q install --upgrade pip
!pip -q install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip -q install pytorch-lightning onnx onnxruntime onnxsim
# onnxruntime-gpu é opcional; ativa se houver GPU compatível
try:
    import torch
    if torch.cuda.is_available():
        !pip -q install onnxruntime-gpu
except Exception as e:
    print("Aviso: onnxruntime-gpu não instalado:", e)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'onnxsim' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'onnxsim'. Discussion can be found at https://github.com/pypa/pip/issues/6334


## 2) Estrutura do projeto e caminhos

In [3]:
from pathlib import Path

ROOT = Path.cwd()
DATA_DIR = ROOT / "data"
MODELS_DIR = ROOT / "models"
ONNX_DIR = ROOT / "onnx"

DATA_DIR.mkdir(exist_ok=True, parents=True)
MODELS_DIR.mkdir(exist_ok=True, parents=True)
ONNX_DIR.mkdir(exist_ok=True, parents=True)

print("ROOT:", ROOT)
print("DATA_DIR:", DATA_DIR)
print("MODELS_DIR:", MODELS_DIR)
print("ONNX_DIR:", ONNX_DIR)

ROOT: /content
DATA_DIR: /content/data
MODELS_DIR: /content/models
ONNX_DIR: /content/onnx


### (Opcional) Montar Google Drive

In [ ]:
USE_GDRIVE = False  # mude para True se quiser salvar no seu Drive
if USE_GDRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    # Ajuste os diretórios, se desejar salvar no Drive:
    # ROOT = Path('/content/drive/MyDrive/seu_projeto')

## 3) Código-fonte — módulos do projeto
Abaixo estão os conteúdos dos arquivos originais gravados como módulos locais para facilitar os imports.

### `data_utils.py`

In [4]:
%%writefile data_utils.py
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from torch.utils.data import DataLoader

def get_dataloaders(model_name: str, dataset_name: str = "ag_news", max_length: int = 128, batch_size: int = 16):
    ds = load_dataset(dataset_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    def tokenize(batch):
        return tokenizer(batch["text"], truncation=True, max_length=max_length)

    tokenized = ds.map(tokenize, batched=True, remove_columns=["text"])
    tokenized = tokenized.rename_column("label", "labels")
    tokenized.set_format(type="torch")

    collator = DataCollatorWithPadding(tokenizer=tokenizer)
    train_dl = DataLoader(tokenized["train"], batch_size=batch_size, shuffle=True, collate_fn=collator)
    test_dl = DataLoader(tokenized["test"], batch_size=batch_size, shuffle=False, collate_fn=collator)

    # Criar validação simples a partir do train (pequeno split)
    val_size = min(4000, len(tokenized["train"]))
    val_subset = torch.utils.data.Subset(tokenized["train"], range(val_size))
    val_dl = DataLoader(val_subset, batch_size=batch_size, shuffle=False, collate_fn=collator)

    return train_dl, val_dl, test_dl, tokenizer

Writing data_utils.py


### `model.py`

In [5]:
%%writefile model.py
from typing import Any, Dict
import torch
import pytorch_lightning as pl
from torch.optim import AdamW
from transformers import AutoModelForSequenceClassification
from torchmetrics.classification import MulticlassAccuracy, MulticlassF1Score

class TextClassifier(pl.LightningModule):
    def __init__(self, model_name: str, num_labels: int = 4, lr: float = 5e-5, weight_decay: float = 0.01):
        super().__init__()
        self.save_hyperparameters()
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
        self.acc = MulticlassAccuracy(num_classes=num_labels, average="macro")
        self.f1 = MulticlassF1Score(num_classes=num_labels, average="macro")

    def forward(self, **batch):
        return self.model(**batch)

    def training_step(self, batch: Dict[str, torch.Tensor], batch_idx: int):
        out = self(**batch)
        preds = out.logits.argmax(dim=-1)
        acc = self.acc(preds, batch["labels"])
        f1 = self.f1(preds, batch["labels"])
        self.log_dict({"train_loss": out.loss, "train_acc": acc, "train_f1": f1}, prog_bar=True, on_step=True, on_epoch=True)
        return out.loss

    def validation_step(self, batch: Dict[str, torch.Tensor], batch_idx: int):
        out = self(**batch)
        preds = out.logits.argmax(dim=-1)
        acc = self.acc(preds, batch["labels"])
        f1 = self.f1(preds, batch["labels"])
        self.log_dict({"val_loss": out.loss, "val_acc": acc, "val_f1": f1}, prog_bar=True, on_step=False, on_epoch=True)

    def test_step(self, batch: Dict[str, torch.Tensor], batch_idx: int):
        out = self(**batch)
        preds = out.logits.argmax(dim=-1)
        acc = self.acc(preds, batch["labels"])
        f1 = self.f1(preds, batch["labels"])
        self.log_dict({"test_loss": out.loss, "test_acc": acc, "test_f1": f1}, prog_bar=True, on_step=False, on_epoch=True)

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=self.hparams.lr, weight_decay=self.hparams.weight_decay)

    def on_train_end(self) -> None:
        # Salva pesos HF finetunados para export ONNX posterior
        self.model.save_pretrained("artifacts/hf")

Writing model.py


### `train.py`

In [6]:
%%writefile train.py
import os
import random
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint
import hydra
from omegaconf import DictConfig, OmegaConf

from src.data_utils import get_dataloaders
from src.model import TextClassifier

def maybe_wandb_logger(cfg):
    use_wb = bool(cfg.logging.get("use_wandb", False))
    if use_wb and os.environ.get("WANDB_API_KEY"):
        import wandb
        from pytorch_lightning.loggers import WandbLogger
        wandb.login()
        return WandbLogger(project=cfg.logging.get("project", "mlops-trilha-minima"))
    return CSVLogger(save_dir="logs", name="runs")

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    pl.seed_everything(seed, workers=True)

@hydra.main(version_base=None, config_path="../configs", config_name="config")
def main(cfg: DictConfig):
    print("Config:\n", OmegaConf.to_yaml(cfg))
    set_seed(cfg.seed)

    train_dl, val_dl, test_dl, tokenizer = get_dataloaders(
        model_name=cfg.model.name,
        dataset_name=cfg.data.dataset_name,
        max_length=cfg.data.max_length,
        batch_size=cfg.data.batch_size,
    )

    model = TextClassifier(
        model_name=cfg.model.name,
        num_labels=cfg.model.num_labels,
        lr=cfg.model.lr,
        weight_decay=cfg.model.weight_decay,
    )

    logger = maybe_wandb_logger(cfg)
    ckpt = ModelCheckpoint(monitor="val_f1", mode="max", save_top_k=1, dirpath="artifacts", filename="model")

    trainer = pl.Trainer(
        max_epochs=cfg.trainer.max_epochs,
        devices=cfg.trainer.devices,
        precision=cfg.trainer.precision,
        logger=logger,
        log_every_n_steps=cfg.trainer.log_every_n_steps,
        callbacks=[ckpt],
    )

    trainer.fit(model, train_dl, val_dl)
    trainer.test(model, test_dl, ckpt_path=ckpt.best_model_path if ckpt.best_model_path else None)
    print(f"Best checkpoint: {ckpt.best_model_path}")

if __name__ == "__main__":
    main()

Writing train.py


### `export_onnx.py`

In [7]:
%%writefile export_onnx.py
import os
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

def export(model_dir="artifacts/hf", out_path="artifacts/model.onnx", opset=13):
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    model = AutoModelForSequenceClassification.from_pretrained(model_dir)
    model.eval()

    dummy = tokenizer("hello world", return_tensors="pt", truncation=True, max_length=128)
    inputs = (dummy["input_ids"], dummy["attention_mask"])
    dynamic_axes = {"input_ids": {0: "batch", 1: "sequence"}, "attention_mask": {0: "batch", 1: "sequence"}, "logits": {0: "batch"}}

    with torch.no_grad():
        torch.onnx.export(
            model,
            args=inputs,
            f=out_path,
            input_names=["input_ids", "attention_mask"],
            output_names=["logits"],
            dynamic_axes=dynamic_axes,
            opset_version=opset,
        )
    print(f"Exported ONNX to {out_path}")

if __name__ == "__main__":
    export()

Writing export_onnx.py


### `infer.py`

In [8]:
%%writefile infer.py
import argparse
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F

def predict(text: str, model_dir: str = "artifacts/hf"):
    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    model = AutoModelForSequenceClassification.from_pretrained(model_dir)
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128)
    with torch.no_grad():
        out = model(**inputs).logits
        probs = F.softmax(out, dim=-1).squeeze().tolist()
        pred = int(out.argmax(dim=-1).item())
    return {"pred": pred, "probs": probs}

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--text", type=str, required=True)
    args = parser.parse_args()
    print(predict(args.text))

Writing infer.py


### `infer_onnx.py`

In [9]:
%%writefile infer_onnx.py
import argparse
import numpy as np
import onnxruntime as ort
from transformers import AutoTokenizer
import torch.nn.functional as F
import torch

def predict(text: str, model_path: str = "artifacts/model.onnx", tokenizer_dir: str = "artifacts/hf"):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir)
    sess = ort.InferenceSession(model_path, providers=["CPUExecutionProvider"])
    enc = tokenizer(text, return_tensors="np", truncation=True, max_length=128)
    inputs = {"input_ids": enc["input_ids"], "attention_mask": enc["attention_mask"]}
    logits = sess.run(["logits"], inputs)[0]
    logits_t = torch.from_numpy(logits)
    probs = F.softmax(logits_t, dim=-1).squeeze().tolist()
    pred = int(np.argmax(logits, axis=-1).item())
    return {"pred": pred, "probs": probs}

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--text", type=str, required=True)
    args = parser.parse_args()
    print(predict(args.text))

Writing infer_onnx.py


## 4) Treinamento (PyTorch)

In [12]:
!pip install hydra-core omegaconf

In [16]:
# ==== Treino isolado (Hydra) — robusto para Colab ====
import os, sys, subprocess, shutil
from pathlib import Path

ROOT = Path.cwd()
SRC  = ROOT / "src"
CONF = ROOT / "configs"
MODELS_DIR = ROOT / "models"
DATA_DIR   = ROOT / "data"

# 0) Dependências necessárias
subprocess.run([sys.executable, "-m", "pip", "install", "-q",
                "pytorch-lightning", "omegaconf", "hydra-core",
                "transformers", "datasets", "torchmetrics"], check=False)

# 1) Estrutura de pacotes
SRC.mkdir(exist_ok=True)
(MODELS_DIR).mkdir(exist_ok=True)
(DATA_DIR).mkdir(exist_ok=True)
(SRC / "__init__.py").write_text("", encoding="utf-8")

# Copia módulos para o pacote src/
def cp(src_name, dst_rel):
    if Path(src_name).exists():
        shutil.copyfile(src_name, SRC / dst_rel)

cp("data_utils.py", "data_utils.py")
cp("model.py", "model.py")

# Mover/copiar o train.py para dentro de src/ (para que config_path ../configs => /content/configs)
if Path("train.py").exists():
    shutil.copyfile("train.py", SRC / "train.py")

# 2) Config Hydra (o decorator aponta para ../configs)
CONF.mkdir(exist_ok=True)
cfg_path = CONF / "config.yaml"
if not cfg_path.exists():
    cfg_path.write_text(
        """# Config mínima para o seu script
seed: 42

logging:
  use_wandb: false

data:
  dataset_name: ag_news
  max_length: 128
  batch_size: 16

model:
  name: distilbert-base-uncased
  num_labels: 4
  lr: 5e-5
  weight_decay: 0.01

trainer:
  max_epochs: 1          # aumente depois
  devices: 1
  precision: 32
  log_every_n_steps: 10
""",
        encoding="utf-8"
    )

# 3) Ambiente
os.environ["PYTHONPATH"] = f"{str(ROOT)}" + (":" + os.environ["PYTHONPATH"] if "PYTHONPATH" in os.environ else "")
os.environ["HYDRA_FULL_ERROR"] = "1"

print("ROOT      :", ROOT)
print("SRC       :", SRC)
print("CONFIGS   :", CONF)
print("DATA_DIR  :", DATA_DIR)
print("MODELS_DIR:", MODELS_DIR)

# 4) Execução isolada (sem herdar os -f do Jupyter)
#    Mantemos o diretório de execução e desativamos a subpasta .hydra
cmd = [sys.executable, "-m", "src.train", "hydra.run.dir=.", "hydra.output_subdir=null"]

print("\n> Executando:", " ".join(cmd), "\n")
try:
    subprocess.run(cmd, check=True)
    print("\n✅ Treinamento finalizado com sucesso.")
except subprocess.CalledProcessError as e:
    # Se falhar, roda capturando stdout/err e exibe tudo
    print("❌ Falha no treinamento. Reexecutando para capturar logs...\n")
    r = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    print(r.stdout)
    raise

ROOT      : /content
SRC       : /content/src
CONFIGS   : /content/configs
DATA_DIR  : /content/data
MODELS_DIR: /content/models

> Executando: /usr/bin/python3 -m src.train hydra.run.dir=. hydra.output_subdir=null 


✅ Treinamento finalizado com sucesso.


## 5) Exportação para ONNX

In [ ]:
# Exportar o modelo treinado para ONNX, usando o script `export_onnx.py`
import os, runpy

env = os.environ.copy()
env["MODELS_DIR"] = str(MODELS_DIR)
env["ONNX_DIR"] = str(ONNX_DIR)

runpy.run_path("export_onnx.py", run_name="__main__")
print("\nArquivos ONNX em:", list(ONNX_DIR.glob("*.onnx")))

## 6) Inferência (PyTorch)

In [ ]:
# Executa inferência usando PyTorch (script `infer.py`)
import os, runpy

env = os.environ.copy()
env["MODELS_DIR"] = str(MODELS_DIR)
env["DATA_DIR"] = str(DATA_DIR)

runpy.run_path("infer.py", run_name="__main__")

## 7) Inferência (ONNX Runtime)

In [ ]:
# Executa inferência usando ONNX Runtime (script `infer_onnx.py`)
import os, runpy

env = os.environ.copy()
env["ONNX_DIR"] = str(ONNX_DIR)
env["DATA_DIR"] = str(DATA_DIR)

runpy.run_path("infer_onnx.py", run_name="__main__")



## 8) Troubleshooting (erros comuns)
- **ModuleNotFoundError**: confirme as instalações na célula de dependências; rode-a novamente.
- **ImportError relativo**: como gravamos os módulos com os mesmos nomes, os imports devem funcionar. Se houver pacotes/paths personalizados no seu projeto, ajuste `sys.path` no início do notebook.
- **CUDA/versão do Torch**: caso a GPU do Colab não esteja ativa, o PyTorch instalará uma versão CPU. Ative a GPU em *Runtime → Change runtime type*.
- **ONNX opset**: se seu export esperar um `opset_version` específico, ajuste no `export_onnx.py`.
